In [14]:
from feature_creation import fc
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer

import lightning.pytorch as pl

from torch.utils.data import DataLoader

import pandas as pd


def build_training_data(config) -> pd.DataFrame:
    df = pd.read_csv(config["training_data_path"])
    df.rename(columns={"period": "date"}, inplace=True)
    df["date"] = pd.to_datetime(df["date"])

    df = fc(df, config["time_windows"], config["large_time_windows"])

    return df


def build_time_series_ds(config) -> DataLoader:

    df = build_training_data(config)

    training_cutoff = round(df["time_idx"].quantile(config["training_cutoff_quantile"]))
    time_varying_unknown_reals = [x for x in df.columns if "rolling" in x] + [config["target"]]
    print(config["time_varying_known_real_additions"])
    time_varying_known_reals = (
        [x for x in df.columns if x.startswith("sin") or x.startswith("cos")]
        + [x for x in df.columns if "shifted" in x]
        + config["time_varying_known_real_additions"]
    )
    ds_train = TimeSeriesDataSet(
        df[lambda x: x.time_idx <= training_cutoff],
        time_idx="time_idx",
        target=config["target"],
        group_ids=config["group_ids"],
        max_encoder_length=config["max_encoder_len"],
        min_prediction_length=1,
        max_prediction_length=config["max_pred_len"],
        static_categoricals=config["static_categoricals"],
        time_varying_known_reals=time_varying_known_reals,
        time_varying_unknown_reals=time_varying_unknown_reals,
        target_normalizer=GroupNormalizer(groups=["fueltype"], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
    )
    ds_val = TimeSeriesDataSet.from_dataset(
        ds_train, df, predict=True, stop_randomization=True
    )  # allow_missing_timesteps=True)

    train_dataloader = ds_train.to_dataloader(train=True, batch_size=config["batch_size"])
    val_dataloader = ds_val.to_dataloader(train=False, batch_size=config["batch_size"] * 10)

    return train_dataloader, val_dataloader, ds_train


def build_tft(config, ds_train) -> TemporalFusionTransformer:
    trainer = pl.Trainer(**config["trainer_params"])

    tft = TemporalFusionTransformer.from_dataset(ds_train, **config["tft_params"])

    return trainer, tft

In [15]:
from config import get_config

config = get_config()

train_dataloader, val_dataloader, ds_train = build_time_series_ds(config)

trainer, g = build_tft(config,ds_train)

['year']


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
/home/luke/.local/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/luke/.local/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [16]:
g.hparams

"attention_head_size":               4
"categorical_groups":                {}
"causal_attention":                  True
"dropout":                           0.4
"embedding_labels":                  {'fueltype': {'COL': 0, 'NG': 1, 'NUC': 2, 'OIL': 3, 'OTH': 4, 'SUN': 5, 'WAT': 6, 'WND': 7}}
"embedding_paddings":                []
"embedding_sizes":                   {'fueltype': (8, 5)}
"hidden_continuous_size":            10
"hidden_continuous_sizes":           {}
"hidden_size":                       32
"learning_rate":                     0.005
"log_gradient_flow":                 False
"log_interval":                      10
"log_val_interval":                  10
"logging_metrics":                   ModuleList(
  (0): SMAPE()
  (1): MAE()
  (2): RMSE()
  (3): MAPE()
)
"loss":                              QuantileLoss(quantiles=[0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98])
"lstm_layers":                       1
"max_encoder_length":                672
"monotone_constaints":             

## find difference between BaseModel and BaseModelWithCovariates

In [3]:
from pytorch_forecasting.models import BaseModel, BaseModelWithCovariates

In [5]:
base_attrs = set(dir(BaseModel))
cov_attrs = set(dir(BaseModelWithCovariates))

In [9]:
base_attrs

{'CHECKPOINT_HYPER_PARAMS_KEY',
 'CHECKPOINT_HYPER_PARAMS_NAME',
 'CHECKPOINT_HYPER_PARAMS_SPECIAL_KEY',
 'CHECKPOINT_HYPER_PARAMS_TYPE',
 'T_destination',
 '_LightningModule__check_allowed',
 '_LightningModule__check_not_nested',
 '_LightningModule__to_tensor',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__jit_unused_properties__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_apply_batch_transfer_handler',
 '_call_batch_hook',
 '_call_impl',
 '_compiled_call_impl',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_jit_is_scripting',
 '_load_from_state_dict',
 '_log_dict_through_fabric',
 '_may

In [11]:
cov_attrs 

{'CHECKPOINT_HYPER_PARAMS_KEY',
 'CHECKPOINT_HYPER_PARAMS_NAME',
 'CHECKPOINT_HYPER_PARAMS_SPECIAL_KEY',
 'CHECKPOINT_HYPER_PARAMS_TYPE',
 'T_destination',
 '_LightningModule__check_allowed',
 '_LightningModule__check_not_nested',
 '_LightningModule__to_tensor',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__jit_unused_properties__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_apply_batch_transfer_handler',
 '_call_batch_hook',
 '_call_impl',
 '_compiled_call_impl',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_jit_is_scripting',
 '_load_from_state_dict',
 '_log_dict_through_fabric',
 '_may

In [6]:
base_custom_attrs = {attr for attr in base_attrs if not attr.startswith("__") and not attr.endswith("__")}
covariate_custom_attrs = {attr for attr in cov_attrs if not attr.startswith("__") and not attr.endswith("__")}

# Identify attributes and methods unique to BaseModelWithCovariates
unique_to_covariate = covariate_custom_attrs - base_custom_attrs

In [7]:
unique_to_covariate

{'calculate_prediction_actual_by_variable',
 'categorical_groups_mapping',
 'categoricals',
 'decoder_variables',
 'encoder_variables',
 'extract_features',
 'plot_prediction_actual_by_variable',
 'reals',
 'static_variables',
 'target_positions'}

In [1]:
from model import build_time_series_ds, build_tft
from config import get_config

cfg = get_config()

train_dataloader, val_dataloader, ds_train = build_time_series_ds(cfg)
trainer, tft = build_tft(cfg, ds_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
/home/luke/.local/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/luke/.local/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [3]:
ds_train.static_reals

['encoder_length', 'value_center', 'value_scale']

In [26]:
tft.hparams.hidden_continuous_sizes.get("value_center", tft.hparams.hidden_continuous_size)

10

# load in mytft

In [1]:
from tft import MyTemporalFusionTransformer
from config import get_config
from model import build_training_data, build_time_series_ds

#import torch

#from pytorch_forecasting import TemporalFusionTransformer


config = get_config()
df, training_cutoff, time_varying_unknown_reals, time_varying_known_reals = build_training_data(config)

train_dataloader, val_dataloader,ds_train = build_time_series_ds(config)

"""
r = TemporalFusionTransformer( 
    max_encoder_length=config["max_encoder_len"],
    static_categoricals=config["static_categoricals"],
    static_reals=config["static_reals"],
    time_varying_reals_encoder=time_varying_known_reals + time_varying_unknown_reals,
    time_varying_reals_decoder=time_varying_known_reals,
    x_reals=config["static_reals"] + time_varying_known_reals + time_varying_unknown_reals,
    x_categoricals=config["group_ids"],
    **config["tft_params"]
)
"""
t = MyTemporalFusionTransformer(
    max_encoder_length=config["max_encoder_len"],
    static_categoricals=config["static_categoricals"],
    static_reals=config["static_reals"],
    time_varying_reals_encoder=time_varying_known_reals + time_varying_unknown_reals,
    time_varying_reals_decoder=time_varying_known_reals,
    x_reals=config["static_reals"] + time_varying_known_reals + time_varying_unknown_reals,
    x_categoricals=config["group_ids"],
    **config["tft_params"]
)

/home/luke/.local/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/luke/.local/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [2]:
i = next(iter(train_dataloader))[0]

In [3]:
for k in i:
    print(k, i[k].shape)

encoder_cat torch.Size([128, 672, 1])
encoder_cont torch.Size([128, 672, 41])
encoder_target torch.Size([128, 672])
encoder_lengths torch.Size([128])
decoder_cat torch.Size([128, 168, 1])
decoder_cont torch.Size([128, 168, 41])
decoder_target torch.Size([128, 168])
decoder_lengths torch.Size([128])
decoder_time_idx torch.Size([128, 168])
groups torch.Size([128, 1])
target_scale torch.Size([128, 2])


In [4]:
i["static_variables"]

KeyError: 'static_variables'

In [5]:
out = t(i)

len of static variables: 4
embeddings_varying_encoder shape: torch.Size([128, 672, 32])
embeddings_varying_decoder shape: torch.Size([128, 168, 32])
static_embedding shape: torch.Size([128, 32])
lstm_output_encoder shape: torch.Size([128, 672, 32])
lstm_output_decoder shape: torch.Size([128, 168, 32])
lstm_output shape: torch.Size([128, 840, 32])
torch.Size([128, 32])
tensor([[ 0.0327,  0.3122,  0.8833,  ...,  0.1788,  0.1305, -1.1699],
        [ 0.3069,  0.3290,  0.3416,  ...,  0.3487, -0.3320, -0.5694],
        [ 0.3516,  0.4481,  0.3561,  ..., -0.3984,  0.0294, -0.7366],
        ...,
        [ 0.2319,  0.5471,  0.4556,  ..., -0.4400, -0.5103, -0.6819],
        [ 0.4475,  1.0339,  0.5467,  ..., -0.0525, -0.5057, -0.7574],
        [ 0.6230,  0.6082,  0.1276,  ...,  0.9015,  0.6393,  0.4197]],
       grad_fn=<SumBackward1>)


all keys

In [6]:
out.static_variables

tensor([[[0.6887, 0.1725, 0.0453, 0.0935]],

        [[0.0535, 0.3311, 0.0671, 0.5482]],

        [[0.0433, 0.4755, 0.0912, 0.3900]],

        [[0.0382, 0.4303, 0.1190, 0.4124]],

        [[0.6485, 0.1619, 0.0384, 0.1513]],

        [[0.2556, 0.1643, 0.0352, 0.5449]],

        [[0.6887, 0.1725, 0.0453, 0.0935]],

        [[0.7048, 0.1209, 0.0433, 0.1310]],

        [[0.2556, 0.1643, 0.0352, 0.5449]],

        [[0.2736, 0.1757, 0.0345, 0.5162]],

        [[0.3444, 0.1978, 0.0334, 0.4244]],

        [[0.6741, 0.1217, 0.0408, 0.1633]],

        [[0.1495, 0.1775, 0.0378, 0.6352]],

        [[0.7656, 0.0831, 0.0635, 0.0878]],

        [[0.0482, 0.3249, 0.0771, 0.5499]],

        [[0.0884, 0.5565, 0.0442, 0.3108]],

        [[0.1327, 0.2891, 0.0371, 0.5412]],

        [[0.2569, 0.0936, 0.0432, 0.6063]],

        [[0.0623, 0.4770, 0.0571, 0.4036]],

        [[0.6443, 0.1589, 0.0381, 0.1587]],

        [[0.6871, 0.1500, 0.0417, 0.1211]],

        [[0.0930, 0.5599, 0.0430, 0.3040]],

        [[

In [7]:
for key in i.keys():
    print(key, i[key].shape)


encoder_cat torch.Size([128, 672, 1])
encoder_cont torch.Size([128, 672, 39])
encoder_target torch.Size([128, 672])
encoder_lengths torch.Size([128])
decoder_cat torch.Size([128, 168, 1])
decoder_cont torch.Size([128, 168, 39])
decoder_target torch.Size([128, 168])
decoder_lengths torch.Size([128])
decoder_time_idx torch.Size([128, 168])
groups torch.Size([128, 1])
target_scale torch.Size([128, 2])


input embeddings

In [8]:
from tft import MyMultiEmbedding

input_embeddings = MyMultiEmbedding(
    embedding_sizes=t.hparams.embedding_sizes,
    categorical_groups=t.hparams.categorical_groups,
    embedding_paddings=t.hparams.embedding_paddings,
    x_categoricals=t.hparams.x_categoricals,
    max_embedding_size=t.hparams.hidden_size,
)

In [9]:
x_cat = torch.cat([i["encoder_cat"], i["decoder_cat"]], dim=1)
print(x_cat.shape)


torch.Size([128, 840, 1])


In [24]:
input_vectors = input_embeddings(x_cat)
print(input_vectors.keys())

dict_keys(['fueltype'])


In [25]:
x_cont = torch.cat([i["encoder_cont"], i["decoder_cont"]], dim=1)
x_cont.shape

torch.Size([128, 840, 39])

In [26]:
input_vectors.update(
    {name: x_cont[..., idx].unsqueeze(-1) for idx, name in enumerate(t.hparams.x_reals) if name in t.reals}
)

In [27]:
for k in input_vectors.keys():
    print(k,input_vectors[k].shape)

fueltype torch.Size([128, 840, 5])
encoder_length torch.Size([128, 840, 1])
value_center torch.Size([128, 840, 1])
value_scale torch.Size([128, 840, 1])
sin_hour torch.Size([128, 840, 1])
cos_hour torch.Size([128, 840, 1])
sin_day_week torch.Size([128, 840, 1])
cos_day_week torch.Size([128, 840, 1])
sin_day_of_year torch.Size([128, 840, 1])
cos_day_of_year torch.Size([128, 840, 1])
sin_week_year torch.Size([128, 840, 1])
cos_week_year torch.Size([128, 840, 1])
sin_month torch.Size([128, 840, 1])
cos_month torch.Size([128, 840, 1])
sin_quarter torch.Size([128, 840, 1])
cos_quarter torch.Size([128, 840, 1])
shifted_24 torch.Size([128, 840, 1])
shifted_48 torch.Size([128, 840, 1])
shifted_168 torch.Size([128, 840, 1])
shifted_730 torch.Size([128, 840, 1])
shifted_8760 torch.Size([128, 840, 1])
year torch.Size([128, 840, 1])
rolling_date_mean_2 torch.Size([128, 840, 1])
rolling_date_std_2 torch.Size([128, 840, 1])
rolling_date_mean_4 torch.Size([128, 840, 1])
rolling_date_std_4 torch.Size(

In [13]:
t.static_variables

['fueltype', 'encoder_length', 'value_center', 'value_scale']

variable selection

output 

In [11]:
o = t(i)

for key in o.keys():
    print(key,o[key].shape)

prediction torch.Size([128, 168, 7])
encoder_attention torch.Size([128, 168, 4, 672])
decoder_attention torch.Size([128, 168, 4, 168])
static_variables torch.Size([128, 1, 4])
encoder_variables torch.Size([128, 672, 1, 35])
encoder_lengths torch.Size([128])
decoder_lengths torch.Size([128])


In [2]:
import numpy as np

logits = np.array([2.0, 1.0, 0.1])
exp_logits = np.exp(logits)
probabilities = exp_logits / np.sum(exp_logits)

In [3]:
exp_logits

array([7.3890561 , 2.71828183, 1.10517092])

In [4]:
probabilities

array([0.65900114, 0.24243297, 0.09856589])